In [4]:
import pandas as pd
from backend.src.utils import MySQLAgent, read_config

import json
from datetime import datetime

import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

In [5]:
year_region = 5

In [6]:
year_region

5

In [7]:
configs = read_config(path="./backend/.env/connections.json")
time_config = 'past'

In [8]:
try:
    #TODO: switch to CrawlerDB
    job_configs = configs["CREDITREPORT"]['Crawler_mysql_conn_info']
    sql_agent = MySQLAgent(job_configs)

    query = f"""
    select * from w_yfy_crd_pst_f
    where debtor_accounting_no = '86156446'
    """
    
    df_pst = sql_agent.read_table(query=query)

    df_pst = df_pst[~df_pst['register_no'].isnull()]


    df_pst['agreement_end_date'] = pd.to_datetime(df_pst['agreement_end_date'])

    current_year = datetime.now().year

    def sliced_data(df):
        df['agreement_end_year'] = df['agreement_end_date'].dt.year
        years_range = list(range(current_year - year_region, current_year))
        df_sliced = df[df['agreement_end_year'].isin(years_range)]
        return df_sliced
    
    if time_config == 'past':
        df_sliced = sliced_data(df_pst)
        if df_sliced['agreement_end_date'].max() < datetime.now():
            # if df_sliced.empty:
                # return {"message": self.no_data_msg}

            # for past : max
            nearest_end_date = df_sliced['agreement_end_date'].max().dt.date
        # elif df_sliced['agreement_end_date'].max() > datetime.now():
            # return {"message": self.no_data_msg}, None, None
        
    elif time_config == 'future':
        df_sliced = sliced_data(df_pst)
        if df_pst['agreement_end_date'].min() >= datetime.now():
            
            # if df_sliced.empty:
                # return {"message": self.no_data_msg}
            # for future: min
            nearest_end_date = df_sliced['agreement_end_date'].min().dt.date

        # elif df_pst['agreement_end_date'].min() < datetime.now():
            # return {"message": self.no_data_msg}, None, None
        
    # else:
        # return {"message": 'wrong time config for pst_analysis'}

except Exception as e:
    error_message = str(e)
    # return {"message": "An error occurred while fetching data: " + error_message}, None, None


In [9]:
def convert_to_int(value):
    clean_value = value.replace('(新台幣)', '').replace(',','').strip()
    return clean_value

df_sliced['agreement_amount'] = df_sliced['agreement_amount'].apply(convert_to_int)

NameError: name 'df_sliced' is not defined

In [ ]:
df_sliced_clean = df_sliced[df_sliced['agreement_amount']!=""].copy()
df_sliced_clean['agreement_amount'] = df_sliced_clean['agreement_amount'].astype(int)
df_sliced_clean['agreement_end_year'] = df_sliced_clean['agreement_end_year'].astype(int)

In [ ]:
df_sliced_clean[['object_type']]

,object_type
1,
2,


In [ ]:
# agreement_amount by year
annual_agreement_aggregates = df_sliced_clean.groupby('agreement_end_year')['agreement_amount'].agg(
    total_agreement_amount='sum',
    average_agreement_amount='mean',
    agreement_count='count'
).reset_index()

# Round the average_agreement_amount to 2 decimal places
annual_agreement_aggregates['average_agreement_amount'] = annual_agreement_aggregates['average_agreement_amount'].round()

annual_agreement_aggregates


,agreement_end_year,total_agreement_amount,average_agreement_amount,agreement_count
0,2023,44482400,22241200.0,2


In [ ]:
df_object_count = df_sliced_clean.groupby('agreement_end_year')['object_type'].agg(object_counts='size').reset_index()

In [ ]:
overall_object_type_counts = {type: df_sliced['object_type'].count() for type in set(df_sliced['object_type'])}
annual_object_type_counts = df_sliced_clean.groupby('agreement_end_year')['object_type'].agg(object_counts='size').reset_index()

if pd.notna(nearest_end_date):
    nearest_end_date_str = nearest_end_date.strftime('%Y-%m-%d')
else:
    nearest_end_date_str = None

pst_dict = {
    "time_config": time_config,
    "nearest_end_date": nearest_end_date_str,
    "annual_agreement_aggregates": annual_agreement_aggregates.to_dict(orient='records'),
    "overall_type_counts": overall_object_type_counts,
    "annual_type_counts": annual_object_type_counts.to_dict(orient='records')
}

In [ ]:
pst_dict

{'time_config': 'past',
 'nearest_end_date': '2023-06-24',
 'annual_agreement_aggregates': [{'agreement_end_year': 2023,
   'total_agreement_amount': 44482400,
   'average_agreement_amount': 22241200.0,
   'agreement_count': 2}],
 'overall_type_counts': {'': 4},
 'annual_object_counts': [{'agreement_end_year': 2023, 'object_counts': 2}]}